In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
import pickle
import random
from multiprocessing import Pool
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import tqdm
from tqdm.contrib.concurrent import process_map

from generator import generate_tracks_paper, generate_tracks_time_varying, generate_tracks_sudden_birth
from runners import generate_tests, run_test_from_config, save_box_whisker_plot
from test_case import TestUseCase2D

from gmphd_fusion.data import StateVector, CovarianceMatrix
from gmphd_fusion.gm import Gaussian, GaussianMixture
from gmphd_fusion.measurement_model import LinearCoordinateMeasurementModel
from gmphd_fusion.motion_models import ConstantVelocityMotionModel

In [3]:
plt.style.use("bmh")
plt.rcParams.update({"figure.figsize": (24, 12),
                     "axes.facecolor": "white",
                     "axes.edgecolor": "black"})

# Define testing use cases

In [4]:
N_RUNS = 100
EXPERIMENTS_DIR = Path("../experiments")

# Change of parameters that we are going to measure
TEST_PARAMS = dict(
    clutter_rate=[5. * i for i in range(11)],
    detection_prob=[0.7 + 0.05 * i for i in range(7)],
    merge_threshold=[4, 10, 20, 50],
    prune_threshold=[1e-6, 1e-5, 1e-4, 1e-3],
    survival_prob=[0.7 + 0.05 * i for i in range(7)],
)

TEST_CASES = dict(
    two_obj_cross=TestUseCase2D(
        surveillance_region=((-1000., -1000.), (1000., 1000.)),
        clutter_rate=50,
        detection_prob=0.98,
        survival_prob=0.99,
        prune_threshold=1e-5,
        merge_threshold=4,
        max_components=1000,
        init_gm=GaussianMixture(),
        birth_gm=GaussianMixture(
            gaussians=[
                Gaussian(
                    mean=StateVector([250, 250, 0, 0]),
                    cov=CovarianceMatrix(np.diag([100, 100, 25, 25])),
                    label=Gaussian.BIRTH_LABEL),
                Gaussian(
                    mean=StateVector([-250, -250, 0, 0]),
                    cov=CovarianceMatrix(np.diag([100, 100, 25, 25])),
                    label=Gaussian.BIRTH_LABEL),
            ],
            weights=[0.1, 0.1],
        ),
        motion_model=ConstantVelocityMotionModel(motion_noise=5),
        measurement_model=LinearCoordinateMeasurementModel(dim_measurement=2, dim_state=4, measurement_noise=10),
        tracks_true=generate_tracks_paper(),
        cpep_radius=20,
        target_weight_threshold=0.5,
    ),
    birth_death_vary=TestUseCase2D(
        surveillance_region=((-1000., -1000.), (1000., 1000.)),
        clutter_rate=50,
        detection_prob=0.98,
        survival_prob=0.99,
        prune_threshold=1e-5,
        merge_threshold=4,
        max_components=1000,
        init_gm=GaussianMixture(),
        birth_gm=GaussianMixture(
            gaussians=[
                Gaussian(
                    mean=StateVector([-1000, 750, 0, 0]),
                    cov=CovarianceMatrix(np.diag([100, 100, 30, 30])),
                    label=Gaussian.BIRTH_LABEL),
                Gaussian(
                    mean=StateVector([1000, -750, 0, 0]),
                    cov=CovarianceMatrix(np.diag([100, 100, 30, 30])),
                    label=Gaussian.BIRTH_LABEL),
            ],
            weights=[0.1, 0.1],
        ),
        motion_model=ConstantVelocityMotionModel(motion_noise=5),
        measurement_model=LinearCoordinateMeasurementModel(dim_measurement=2, dim_state=4, measurement_noise=10),
        tracks_true=generate_tracks_time_varying(),
        cpep_radius=20,
        target_weight_threshold=0.5,
    ),
    birth_no_fusion=TestUseCase2D(
        surveillance_region=((-1000., -1000.), (1000., 1000.)),
        clutter_rate=50,
        detection_prob=0.98,
        survival_prob=0.99,
        prune_threshold=1e-5,
        merge_threshold=4,
        max_components=1000,
        init_gm=GaussianMixture(),
        birth_gm=GaussianMixture(
            gaussians=[
                Gaussian(
                    mean=StateVector([-1000, 750, 0, 0]),
                    cov=CovarianceMatrix(np.diag([100, 100, 30, 30])),
                    label=Gaussian.BIRTH_LABEL),
                Gaussian(
                    mean=StateVector([1000, -750, 0, 0]),
                    cov=CovarianceMatrix(np.diag([100, 100, 30, 30])),
                    label=Gaussian.BIRTH_LABEL),
            ],
            weights=[0.1, 0.1],
        ),
        motion_model=ConstantVelocityMotionModel(motion_noise=5),
        measurement_model=LinearCoordinateMeasurementModel(dim_measurement=2, dim_state=4, measurement_noise=10),
        tracks_true=generate_tracks_sudden_birth(),
        cpep_radius=20,
        target_weight_threshold=0.5,
    ),
    birth_fusion=TestUseCase2D(
        surveillance_region=((-1000., -1000.), (1000., 1000.)),
        clutter_rate=50,
        detection_prob=0.98,
        survival_prob=0.99,
        prune_threshold=1e-5,
        merge_threshold=4,
        max_components=1000,
        init_gm=GaussianMixture(),
        birth_gm=GaussianMixture(
            gaussians=[
                Gaussian(
                    mean=StateVector([-1000, 750, 0, 0]),
                    cov=CovarianceMatrix(np.diag([100, 100, 30, 30])),
                    label=Gaussian.BIRTH_LABEL),
                Gaussian(
                    mean=StateVector([1000, -750, 0, 0]),
                    cov=CovarianceMatrix(np.diag([100, 100, 30, 30])),
                    label=Gaussian.BIRTH_LABEL),
            ],
            weights=[0.1, 0.1],
        ),
        motion_model=ConstantVelocityMotionModel(motion_noise=5),
        measurement_model=LinearCoordinateMeasurementModel(dim_measurement=2, dim_state=4, measurement_noise=10),
        tracks_true=generate_tracks_sudden_birth(),
        fuse={
            20: GaussianMixture(
                gaussians=[
                    Gaussian(StateVector([-1050, -1050, 25, 25]), cov=CovarianceMatrix(np.diag([100, 100, 10, 10])),
                             label=None)],
                weights=[0.6],
            ),
            60: GaussianMixture(
                gaussians=[Gaussian(StateVector([1020, 1020, -22, -22]), cov=CovarianceMatrix(np.diag([40, 40, 5, 5])),
                                    label=None)],
                weights=[0.7],
            ),
        },
        cpep_radius=20,
        target_weight_threshold=0.5,
    ),
)

# Run

In [5]:
RUN = False

run_configurations = generate_tests(N_RUNS, TEST_CASES, TEST_PARAMS, EXPERIMENTS_DIR)
print(f"Total number of test configurations: {len(run_configurations)}")

Total number of test configurations: 13200


In [6]:
%%time
if RUN:
    pool = Pool()
    random.shuffle(run_configurations)
    success_flags = process_map(
        run_test_from_config,
        run_configurations,
        tqdm_class=tqdm.notebook.tqdm,
        max_workers=10,
        chunksize=100,
    )
else:
    success_flags = [True] * len(run_configurations)

failed_indices = np.where(~np.array(success_flags))[0]
print(f"Total failed: {len(failed_indices)}")

Total failed: 0
CPU times: user 1.2 ms, sys: 329 µs, total: 1.53 ms
Wall time: 1.27 ms


# Compute metrics and build box plots

In [7]:
def prepare_metrics_struct():
    struct = {}
    for tname in TEST_CASES.keys():
        struct[tname] = dict(
            cpep={
                param_name: {
                    f"{param_val:.3e}": [0.] * N_RUNS
                    for param_val in TEST_PARAMS[param_name]
                }
                for param_name in TEST_PARAMS.keys()
            },
            eae={
                param_name: {
                    f"{param_val:.3e}": [0.] * N_RUNS
                    for param_val in TEST_PARAMS[param_name]
                }
                for param_name in TEST_PARAMS.keys()
            }
        )
    return struct


def _read_metrics(p: Path):
    with (p / "cpep_time.pickle").open("rb") as f:
        cpep = pickle.load(f)
        cpep = np.mean(cpep)
    with (p / "eae.pickle").open("rb") as f:
        eae = pickle.load(f)
    return cpep, eae


def read_saved_metrics(experiments_dir: Path):
    paths = [p.parent for p in experiments_dir.glob("**/_FINISHED")]
    
    metrics_data = prepare_metrics_struct()
    for p in paths:
        # path structure: {test_name}/{param_name}={param_value:.3e}/{index:03d}
        index = int(p.name)
        param_name, param_value = p.parent.name.split("=")
        test_name = p.parent.parent.name
        
        cpep, eae = _read_metrics(p)
        metrics_data[test_name]["cpep"][param_name][param_value][index] = cpep
        metrics_data[test_name]["eae"][param_name][param_value][index] = eae
    
    ret = {}
    for test_name, metrics_dict in metrics_data.items():
        for metric_name, params_dict in metrics_dict.items():
            for param_name, param_values in params_dict.items():
                param_values, runs_data = zip(*param_values.items())
                ret[(test_name, metric_name, param_name)] = {
                    "labels": param_values,
                    "data": runs_data,
                }
    return ret

In [8]:
metrics_data = read_saved_metrics(EXPERIMENTS_DIR)

In [10]:
metrics_mapping = {
    "cpep": {
        "y_label": r'$\mathrm{CPEP}_k(r)\ (\mathrm{time\ averaged})$',
    },
    "eae": {
        "y_label": r'$E\left[\left|{|\hat{X}_k| - |X_k|}\right|\right]\ (\mathrm{time\ averaged})$',
    },
}

param_mapping = {
    "clutter_rate": {
        # "/ 4" is for "/ surveillance_area" but multiplied by the number in the text (10^6)
        "label_func": lambda l: f"{float(l) / 4.:.2f}",
        "x_text": r'$\times\mathdefault{10^{-6}}$',
        "x_label": r'$\lambda_c$',
    },
    "detection_prob": {
        "label_func": lambda l: f"{float(l):.2f}",
        "x_text": None,
        "x_label": r'$P_{D,k}$',
    },
    "prune_threshold": {
        "label_func": lambda l: f"{float(l):.0e}",
        "x_text": None,
        "x_label": r'$\tau$',
    },
    "merge_threshold": {
        "label_func": lambda l: f"{float(l):.0f}",
        "x_text": None,
        "x_label": r'$U$',
    },
    "survival_prob": {
        "label_func": lambda l: f"{float(l):.2f}",
        "x_text": None,
        "x_label": r'$P_{S,k}$',
    },
}

for (test_name, metric_name, param_name), mdata in metrics_data.items():
    x_label = param_mapping[param_name]["x_label"]
    y_label = metrics_mapping[metric_name]["y_label"]
    x_ticks = [param_mapping[param_name]["label_func"](l) for l in mdata["labels"]]
    x_text = param_mapping[param_name]["x_text"]
    experiments = mdata["data"]
    save_dir = EXPERIMENTS_DIR / test_name
    save_box_whisker_plot(x_label, y_label, x_ticks, x_text, experiments, save_dir)

# Quick and dirty

In [31]:
def save_gt_tracks_plot(uc: TestUseCase2D, save_dir: Path):
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.set_xlim((-1100, 1100))
    ax.set_ylim((-1100, 1100))
    ax.set_xlabel("x coordinate (in m)")
    ax.set_ylabel("y coordinate (in m)")

    for t in uc.tracks_true:
        start, end = None, None
        for e in t.estimates:
            if e is not None:
                start = e
                break
        for e in reversed(t.estimates):
            if e is not None:
                end = e
                break

        x = start[0, 0]
        dx = end[0, 0] - start[0, 0]
        y = start[1, 0]
        dy = end[1, 0] - start[1, 0]
        ax.arrow(x, y, dx, dy, width=5, head_width=25, length_includes_head=True, color="black")
        ax.scatter([x], [y], facecolors="black", edgecolors="black", marker="o", s=25)
    fig.savefig(str(save_dir / "true_tracks.png"), bbox_inches="tight")
    plt.close(fig)


for name, uc in TEST_CASES.items():
    save_gt_tracks_plot(uc, EXPERIMENTS_DIR / name)

In [32]:
from gmphd_fusion.visualize import visualize_coord_change


def coordinate_change_plot(uc, coord_idx, coord_name, save_dir):
    fig, ax = plt.subplots(1, 1, figsize=(10, 4))
    ax.set_xlim(0, uc.samples_per_test)
    ax.set_ylim(*uc.ylim)

    visualize_coord_change(ax, uc.tracks_true, coord_idx, coord_name)
    fig.savefig(str(save_dir / f"coord_true_{coord_name}_plot.png"), bbox_inches="tight")
    plt.close(fig)


for name, uc in TEST_CASES.items():
    coordinate_change_plot(uc, 0, "x", EXPERIMENTS_DIR / name)
    coordinate_change_plot(uc, 1, "y", EXPERIMENTS_DIR / name)

In [43]:
for name, uc in TEST_CASES.items():
    print(f"[{name}] Number of true tracks: {len(uc.tracks_true)}")

    ntargets_at_k = list(map(sum, [[1 if t.estimate_at(i) is not None else 0 for t in uc.tracks_true] for i in range(1, 100)]))
    print(f"[{name}] Min targets: {min(ntargets_at_k)}")
    print(f"[{name}] Max targets: {max(ntargets_at_k)}")

    for i, t in enumerate(uc.tracks_true):
        for k in range(0, 100):
            e = t.estimate_at(k)
            if e is not None:
                init_state = e.tolist()
                init_time = k
                break
        print(f"Initial state for track {i} (time k={k}): {init_state}")
    print("-" * 60)

[two_obj_cross] Number of true tracks: 2
[two_obj_cross] Min targets: 2
[two_obj_cross] Max targets: 2
Initial state for track 0 (time k=1): [[250.0], [250.0], [2.5], [-12.0]]
Initial state for track 1 (time k=1): [[-250.0], [-250.0], [12.0], [-2.5]]
------------------------------------------------------------
[birth_death_vary] Number of true tracks: 6
[birth_death_vary] Min targets: 1
[birth_death_vary] Max targets: 5
Initial state for track 0 (time k=0): [[-1000.0], [750.0], [25.0], [0.0]]
Initial state for track 1 (time k=10): [[1000.0], [-750.0], [-25.0], [-0.0]]
Initial state for track 2 (time k=20): [[-1000.0], [750.0], [17.67766952966369], [-17.677669529663685]]
Initial state for track 3 (time k=30): [[1000.0], [-750.0], [-17.67766952966369], [17.677669529663685]]
Initial state for track 4 (time k=40): [[-1000.0], [750.0], [1.5308084989341915e-15], [-25.0]]
Initial state for track 5 (time k=50): [[1000.0], [-750.0], [-1.5308084989341915e-15], [25.0]]
---------------------------

In [12]:
# extract metrics
cases = ["birth_no_fusion", "birth_fusion"]
cnf_path = Path("experiments/birth_no_fusion/clutter_rate=5.000e+01")
cf = Path("experiments/birth_fusion/clutter_rate=5.000e+01")

keys = [
    ("birth_no_fusion", "cpep", "clutter_rate"),
    ("birth_no_fusion", "eae", "clutter_rate"),
    ("birth_fusion", "cpep", "clutter_rate"),
    ("birth_fusion", "eae", "clutter_rate")
]

for key in keys:
    data = metrics_data[key]["data"][-1]
    print(f"[{key[0]}] {key[1].upper()}: {np.mean(data):.5f}")

[birth_no_fusion] CPEP: 0.87876
[birth_no_fusion] EAE: 1.00350
[birth_fusion] CPEP: 0.85900
[birth_fusion] EAE: 0.35380
